In [1]:
from pyspark import SparkContext
from pyspark.sql import types
import pyspark.sql.functions as sf
from pyspark.sql.types import Row

In [2]:

ACCESS_KEY = ""
# Encode the Secret Key as that can contain "/"
SECRET_KEY = ""
ENCODED_SECRET_KEY = SECRET_KEY.replace("/", "%2F")
AWS_BUCKET_NAME = "uwdb"
MOUNT_NAME = "tpch"

#dbutils.fs.mount("s3a://%s:%s@%s" % (ACCESS_KEY, ENCODED_SECRET_KEY, AWS_BUCKET_NAME), "/mnt/%s" % MOUNT_NAME)
#display(dbutils.fs.ls("/mnt/%s" % MOUNT_NAME))

display(dbutils.fs.ls("/mnt/%s/tpch/uniform/100GB" % MOUNT_NAME))

In [3]:
spark.conf.set("spark.sql.qubole.split.computation", True)
spark.conf.set("spark.sql.shuffle.partitions", 10)
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", 300000000)

In [4]:
customer_uniform_100GB_rdd = sc.textFile("/mnt/%s/tpch/uniform/100GB/customer.tbl" % MOUNT_NAME)
customer_uniform_100GB_data_rdd = customer_uniform_100GB_rdd.map(lambda line: [x for x in line.split('|')])
customer_uniform_100GB_df=customer_uniform_100GB_data_rdd.toDF(['CUSTKEY','NAME','ADDRESS','NATIONKEY','PHONE','ACCTBAL','MKTSEGMENT','COMMENT','SKIP'])
customer_uniform_100GB_df.registerTempTable('customer_uniform_100GB')

In [5]:
lineitem_uniform_100GB_rdd = sc.textFile("/mnt/%s/tpch/uniform/100GB/lineitem.tbl" % MOUNT_NAME)
lineitem_uniform_100GB_data_rdd = lineitem_uniform_100GB_rdd.map(lambda line: [x for x in line.split('|')])
lineitem_uniform_100GB_df=lineitem_uniform_100GB_data_rdd.toDF(['OrderKey','PartKey','SuppKey','LineNumber','Quantity','ExtendedPrice','Discount','Tax','ReturnFlag','LineStatus','ShipDate','CommitDate','ReceiptDate','ShipInstruct','ShipMode','Comment','Skip'])
lineitem_uniform_100GB_df.registerTempTable('lineitem_uniform_100GB')

In [6]:
nation_uniform_100GB_rdd = sc.textFile("/mnt/%s/tpch/uniform/100GB/nation.tbl" % MOUNT_NAME)
nation_uniform_100GB_data_rdd = nation_uniform_100GB_rdd.map(lambda line: [x for x in line.split('|')])
nation_uniform_100GB_df=nation_uniform_100GB_data_rdd.toDF(['NationKey','Name','RegionKey','Comment','Skip'])
nation_uniform_100GB_df.registerTempTable('nation_uniform_100GB')

In [7]:
orders_uniform_100GB_rdd = sc.textFile("/mnt/%s/tpch/uniform/100GB/order.tbl" % MOUNT_NAME)
orders_uniform_100GB_data_rdd = orders_uniform_100GB_rdd.map(lambda line: [x for x in line.split('|')])
orders_uniform_100GB_df=orders_uniform_100GB_data_rdd.toDF(['OrderKey','CustKey','OrderStatus','TotalPrice','OrderDate','OrderPriority','Clerk','ShipPriority','Comment','Skip'])
orders_uniform_100GB_df.registerTempTable('orders_uniform_100GB')

In [8]:
part_uniform_100GB_rdd = sc.textFile("/mnt/%s/tpch/uniform/100GB/part.tbl" % MOUNT_NAME)
part_uniform_100GB_data_rdd = part_uniform_100GB_rdd.map(lambda line: [x for x in line.split('|')])
part_uniform_100GB_df=part_uniform_100GB_data_rdd.toDF(['PartKey','Name','Mfgr','Brand','Type','Size','Container','RetailPrice','Comment','Skip'])
part_uniform_100GB_df.registerTempTable('part_uniform_100GB')

In [9]:
partsupp_uniform_100GB_rdd = sc.textFile("/mnt/%s/tpch/uniform/100GB/partsupp.tbl" % MOUNT_NAME)
partsupp_uniform_100GB_data_rdd = partsupp_uniform_100GB_rdd.map(lambda line: [x for x in line.split('|')])
partsupp_uniform_100GB_df=partsupp_uniform_100GB_data_rdd.toDF(['PartKey','SuppKey','AvailQty','SupplyCost','Comment','Skip'])
partsupp_uniform_100GB_df.registerTempTable('partsupp_uniform_100GB')

In [10]:
region_uniform_100GB_rdd = sc.textFile("/mnt/%s/tpch/uniform/100GB/region.tbl" % MOUNT_NAME)
region_uniform_100GB_data_rdd = region_uniform_100GB_rdd.map(lambda line: [x for x in line.split('|')])
region_uniform_100GB_df=region_uniform_100GB_data_rdd.toDF(['RegionKey','Name','Comment','Skip'])
region_uniform_100GB_df.registerTempTable('region_uniform_100GB')

In [11]:
supplier_uniform_100GB_rdd = sc.textFile("/mnt/%s/tpch/uniform/100GB/supplier.tbl" % MOUNT_NAME)
supplier_uniform_100GB_data_rdd = supplier_uniform_100GB_rdd.map(lambda line: [x for x in line.split('|')])
supplier_uniform_100GB_df=supplier_uniform_100GB_data_rdd.toDF(['SuppKey','Name','Address','NationKey','Phone','AcctBal','Comment','Skip'])
supplier_uniform_100GB_df.registerTempTable('supplier_uniform_100GB')

In [12]:
sqlContext.cacheTable("orders_uniform_100GB")
sqlContext.cacheTable("lineitem_uniform_100GB")
cache_sql = sqlContext.sql("SELECT count(*) FROM lineitem_uniform_100GB")
cache_sql.show()
cache_sql = sqlContext.sql("SELECT count(*) FROM orders_uniform_100GB")
cache_sql.show()
cache_sql = sqlContext.sql("SELECT count(*) FROM customer_uniform_100GB")
cache_sql.show()
cache_sql = sqlContext.sql("SELECT count(*) FROM part_uniform_100GB")
cache_sql.show()
cache_sql = sqlContext.sql("SELECT count(*) FROM partsupp_uniform_100GB")
cache_sql.show()

In [13]:
query1=sqlContext.sql("select "
    "ReturnFlag,"
	"LineStatus,"
	"sum(Quantity) as sum_qty,"
	"sum(ExtendedPrice) as sum_base_price,"
	"sum(ExtendedPrice * (1 - Discount)) as sum_disc_price,"
	"sum(ExtendedPrice * (1 - Discount) * (1 + Tax)) as sum_charge,"
	"avg(Quantity) as avg_qty,"
	"avg(ExtendedPrice) as avg_price,"
	"avg(Discount) as avg_disc,"
	"count(*) as count_order "
"from "
	"lineitem_uniform_100GB "
"where "
	"ShipDate <= \'1998-09-16\' "
"group by "
	"ReturnFlag, "
	"LineStatus "
"order by "
	"ReturnFlag, "
	"LineStatus")


In [14]:
query1.show()

In [15]:
query3=sqlContext.sql(
  "select "
	"lineitem_uniform_100GB.OrderKey,"
	"sum(ExtendedPrice * (1 - Discount)) as revenue,"
	"OrderDate,"
	"ShipPriority "
"from "
	"customer_uniform_100GB,"
	"orders_uniform_100GB,"
	"lineitem_uniform_100GB "
"where "
	"MKTSEGMENT = \'BUILDING\' "
	"and customer_uniform_100GB.CUSTKEY = orders_uniform_100GB.CustKey "
	"and lineitem_uniform_100GB.OrderKey = orders_uniform_100GB.OrderKey "
	"and OrderDate < \'1995-03-22\' "
	"and ShipDate > \'1995-03-22\' "
"group by "
	"lineitem_uniform_100GB.OrderKey,"
	"OrderDate,"
	"ShipPriority "
"order by "
	"Revenue desc,"
	"OrderDate "
)

In [16]:
query3.show()

In [17]:
query4=sqlContext.sql(
"select "
	"OrderPriority,"
	"count(*) as order_count "
"from "
	"orders_uniform_100GB as o "
"where "
	"OrderDate >= \'1996-05-01\' "
	"and OrderDate < \'1996-08-01\' "
	"and exists ("
		"select "
			"* "
		"from "
			"lineitem_uniform_100GB "
		"where "
			"lineitem_uniform_100GB.OrderKey = o.OrderKey "
			"and CommitDate < ReceiptDate "
	")"
"group by "
	"OrderPriority "
"order by "
	"OrderPriority"
)

In [18]:
query4.show()

In [19]:
query5=sqlContext.sql(
"select "
	"nation_uniform_100GB.Name,"
	"sum(ExtendedPrice * (1 - Discount)) as revenue "
"from "
	"customer_uniform_100GB,"
	"orders_uniform_100GB,"
	"lineitem_uniform_100GB,"
	"supplier_uniform_100GB,"
	"nation_uniform_100GB,"
	"region_uniform_100GB "
"where "
	"customer_uniform_100GB.CUSTKEY = orders_uniform_100GB.CustKey "
	"and lineitem_uniform_100GB.OrderKey = orders_uniform_100GB.OrderKey "
	"and lineitem_uniform_100GB.Suppkey = supplier_uniform_100GB.Suppkey "
	"and customer_uniform_100GB.NATIONKEY = supplier_uniform_100GB.NationKey "
	"and supplier_uniform_100GB.NationKey = nation_uniform_100GB.NationKey "
	"and nation_uniform_100GB.RegionKey = region_uniform_100GB.RegionKey "
	"and region_uniform_100GB.Name = \'AFRICA\'"
	"and OrderDate >= \'1993-01-01\'"
	"and OrderDate < \'1994-01-01\' "
"group by "
	"nation_uniform_100GB.Name "
"order by "
	"Revenue desc"
)

In [20]:
query5.show()

In [21]:
query6 = sqlContext.sql(
"select "
	"sum(ExtendedPrice * Discount) as revenue "
"from "
	"lineitem_uniform_100GB "
"where "
	"ShipDate >= \'1993-01-01\' "
	"and Shipdate < \'1994-01-01\' "
	"and Discount between 0.06 - 0.01 and 0.06 + 0.01 "
	"and Quantity < 25"
)

In [22]:
query6.show()

In [23]:
query7=sqlContext.sql(
"select "
	"supp_nation,"
	"cust_nation,"
	"l_year,"
	"sum(volume) as revenue "
"from "
	"( "
		"select "
			"n1.Name as supp_nation,"
			"n2.Name as cust_nation,"
			"year(lineitem_uniform_100GB.ShipDate) as l_year,"
			"ExtendedPrice * (1 - Discount) as volume "
		"from "
			"supplier_uniform_100GB,"
			"lineitem_uniform_100GB,"
			"orders_uniform_100GB,"
			"customer_uniform_100GB,"
			"nation_uniform_100GB n1,"
			"nation_uniform_100GB n2 "
		"where "
			"supplier_uniform_100GB.SuppKey = lineitem_uniform_100GB.SuppKey "
			"and orders_uniform_100GB.OrderKey = lineitem_uniform_100GB.OrderKey "
			"and customer_uniform_100GB.CUSTKEY = orders_uniform_100GB.CustKey "
			"and supplier_uniform_100GB.NationKey = n1.NationKey "
			"and customer_uniform_100GB.NationKey = n2.NationKey "
			"and ( "
				"(n1.Name = \'KENYA\' and n2.Name = \'PERU\') "
				"or (n1.Name = \'PERU\' and n2.Name = \'KENYA\') "
			") "
			"and lineitem_uniform_100GB.ShipDate between \'1995-01-01\' and \'1996-12-31\' "
	") as shipping "
"group by "
	"supp_nation, "
	"cust_nation, "
	"l_year "
"order by "
	"supp_nation, "
	"cust_nation, "
	"l_year"
)

In [24]:
query7.show()

In [25]:
query10=sqlContext.sql(
"select "
	"customer_uniform_100GB.CUSTKEY,"
	"customer_uniform_100GB.NAME,"
	"sum(ExtendedPrice * (1 - Discount)) as revenue,"
	"ACCTBAL,"
	"nation_uniform_100GB.Name,"
	"Address,"
	"Phone,"
	"customer_uniform_100GB.Comment "
"from "
	"customer_uniform_100GB,"
	"orders_uniform_100GB,"
	"lineitem_uniform_100GB,"
	"nation_uniform_100GB "
"where "
	"customer_uniform_100GB.CUSTKEY = orders_uniform_100GB.CustKey "
	"and lineitem_uniform_100GB.OrderKey = orders_uniform_100GB.OrderKey "
	"and orders_uniform_100GB.OrderDate >= \'1993-07-01\' "
	"and orders_uniform_100GB.OrderDate < '1993-10-01' "
	"and lineitem_uniform_100GB.ReturnFlag = \'R\' "
	"and customer_uniform_100GB.NationKey = nation_uniform_100GB.NationKey "
"group by "
	"customer_uniform_100GB.CustKey,"
	"customer_uniform_100GB.NAME,"
	"ACCTBAL,"
	"PHONE,"
	"nation_uniform_100GB.name,"
	"ADDRESS,"
	"customer_uniform_100GB.COMMENT "
"order by "
	"Revenue desc"
)

In [26]:
query10.show()

In [27]:
query12=sqlContext.sql(
"select "
	"ShipMode,"
	"sum(case "
		"when OrderPriority = \'1-URGENT\' "
			"or OrderPriority = \'2-HIGH\' "
			"then 1 "
		"else 0 "
	"end) as high_line_count, "
	"sum(case "
		"when OrderPriority <> \'1-URGENT\' "
			"and OrderPriority <> \'2-HIGH\' "
			"then 1 "
		"else 0 "
	"end) as low_line_count "
"from "
	"orders_uniform_100GB,"
	"lineitem_uniform_100GB "
"where "
	"orders_uniform_100GB.OrderKey = lineitem_uniform_100GB.OrderKey "
	"and ShipMode in (\'REG AIR\', \'MAIL\') "
	"and CommitDate < ReceiptDate "
	"and ShipDate < CommitDate "
	"and ReceiptDate >= \'1995-01-01\' "
	"and ReceiptDate < \'1996-01-01\' "
"group by "
    "ShipMode "
"order by "
	"ShipMode"
)

In [28]:
query12.show()

In [29]:
query13=sqlContext.sql(
"select "
	"c_count,"
	"count(*) as custdist "
"from "
	"( "
		"select "
			"customer_uniform_100GB.CUSTKEY, "
			"count(orders_uniform_100GB.OrderKey) as c_count "
		"from "
			"customer_uniform_100GB left outer join orders_uniform_100GB on "
				"customer_uniform_100GB.CUSTKEY = orders_uniform_100GB.CustKey "
				"and orders_uniform_100GB.Comment not like '%unusual%accounts%' "
		"group by "
			"customer_uniform_100GB.CUSTKEY "
	") c_orders "
"group by "
	"c_count "
"order by "
	"custdist desc, "
	"c_count desc "
)

In [30]:
query13.show()

In [31]:
query14=sqlContext.sql(
"select "
	"100.00 * sum(case "
		"when Type like \'PROMO%\' "
			"then ExtendedPrice * (1 - Discount) "
		"else 0 "
	"end) / sum(ExtendedPrice * (1 - Discount)) as promo_revenue "
"from "
	"lineitem_uniform_100GB, "
	"part_uniform_100GB "
"where "
	"lineitem_uniform_100GB.partkey = part_uniform_100GB.partkey "
	"and ShipDate >= \'1995-08-01\' "
	"and ShipDate < \'1995-09-01\' "
)

In [32]:
query14.show()

In [33]:
query16=sqlContext.sql(
"select "
	"Brand, "
	"Type, "
	"Size, "
	"count(distinct partsupp_uniform_100GB.Suppkey) as supplier_cnt "
"from "
	"partsupp_uniform_100GB, "
	"part_uniform_100GB "
"where "
	"part_uniform_100GB.PartKey = partsupp_uniform_100GB.PartKey "
	"and Brand <> 'Brand#34' "
	"and Type not like 'ECONOMY BRUSHED%' "
	"and Size in (22, 14, 27, 49, 21, 33, 35, 28) "
	"and partsupp_uniform_100GB.SuppKey not in ( "
		"select "
			"Supplier_uniform_100GB.SuppKey "
		"from "
			"Supplier_uniform_100GB "
		"where "
			"Supplier_uniform_100GB.Comment like '%Customer%Complaints%' "
	") "
"group by "
	"Brand, "
	"Type, "
	"Size "
"order by "
	"supplier_cnt desc, "
	"Brand, "
	"Type, "
	"Size "
)

In [34]:
query16.show()

In [35]:
query18=sqlContext.sql(
"with q17_part as ( "
  "select PartKey from part_uniform_100GB where "
  "Brand = \'Brand#23\' "
  "and Container = \'MED BOX\' "
"), "
"q17_avg as ( "
  "select lineitem_uniform_100GB.PartKey as t_partkey, 0.2 * avg(Quantity) as t_avg_quantity "
  "from lineitem_uniform_100GB "
  "where lineitem_uniform_100GB.PartKey IN (select q17_part.PartKey from q17_part) "
  "group by lineitem_uniform_100GB.PartKey "
"), "
"q17_price as ( "
  "select "
  "Quantity,"
  "lineitem_uniform_100GB.PartKey, "
  "ExtendedPrice "
  "from "
  "lineitem_uniform_100GB "
  "where "
  "lineitem_uniform_100GB.PartKey IN (select PartKey from q17_part) "
") "
"select cast(sum(ExtendedPrice) / 7.0 as decimal(32,2)) as avg_yearly "
"from q17_avg, q17_price "
"where "
"t_partkey = PartKey and Quantity < t_avg_quantity "
)

In [36]:
query18.show()

In [37]:
quey19=sqlContext.sql(
"select "
	"sum(ExtendedPrice* (1 - Discount)) as revenue "
"from "
	"lineitem_uniform_100GB, "
	"part_uniform_100GB "
"where "
	"("
		"part_uniform_100GB.partkey = lineitem_uniform_100GB.partkey "
		"and Brand = \'Brand#32\' "
		"and Container in (\'SM CASE\', \'SM BOX\', \'SM PACK\', \'SM PKG\') "
		"and Quantity >= 7 and Quantity <= 7 + 10 "
		"and Size between 1 and 5 "
		"and ShipMode in (\'AIR\', \'AIR REG\') "
		"and ShipInstruct = \'DELIVER IN PERSON\' "
	") "
	"or "
	"( "
		"part_uniform_100GB.partkey = lineitem_uniform_100GB.partkey "
		"and Brand = \'Brand#35\' "
		"and Container in (\'MED BAG\', \'MED BOX\', \'MED PKG\', \'MED PACK\') "
		"and Quantity >= 15 and Quantity <= 15 + 10 "
		"and Size between 1 and 10 "
		"and ShipMode in (\'AIR\', \'AIR REG\') "
		"and ShipInstruct = 'DELIVER IN PERSON' "
	") "
	"or "
	"( "
		"part_uniform_100GB.partkey = lineitem_uniform_100GB.partkey "
		"and Brand = \'Brand#24\' "
		"and Container in (\'LG CASE\', \'LG BOX\', \'LG PACK\', \'LG PKG\') "
		"and Quantity >= 26 and Quantity <= 26 + 10 "
		"and Size between 1 and 15 "
		"and ShipMode in (\'AIR\', \'AIR REG\') "
		"and ShipInstruct = \'DELIVER IN PERSON' "
	") "
)

In [38]:
query19.show()